In [1]:
# import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


D:\ShipSupplies\DA\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install transformers

In [3]:
# !pip install torch

In [4]:
# import tensorflow as tf

# # GPU 디바이스 목록 가져오기
# gpu_devices = tf.config.list_physical_devices('GPU')

# if len(gpu_devices) > 0:
#     print("사용 가능한 GPU가 있습니다.")
#     for device in gpu_devices:
#         print("GPU 디바이스 이름:", device.name)
# else:
#     print("사용 가능한 GPU가 없습니다.")


In [5]:
# !pip install bert-for-tf2
# !pip install tensorflow_hub

In [6]:
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [7]:
data = pd.read_csv('../data/raw_postpro.csv') # 오류가나면 추가해주세요 .encoding = 'cp949'
# 컬럼 삭제
df = data.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   key2         20517 non-null  object
 8   발주처          20517 non-null  object
 9   D/T          20517 non-null  object
 10  Control No.  20517 non-null  object
 11  leadtime     20517 non-null  int64 
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [9]:
df = df[['청구품목','발주처','Machinery', 'Assembly' , "key1",'key2']]
# 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처'

In [10]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [11]:
df = df.drop(['key2'], axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   청구품목          20517 non-null  object
 1   발주처           20517 non-null  object
 2   Machinery     20517 non-null  object
 3   Assembly      20517 non-null  object
 4   key1          20517 non-null  object
 5   key2_encoded  20517 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 881.7+ KB


In [13]:
# text = df[[ '청구품목', '발주처','Machinery', 'Assembly' , "key1"]].apply(lambda row: ' '.join(row), axis=1)
df_text = df[['청구품목', '발주처', 'Machinery', 'Assembly', 'key1']].apply(lambda row: ' '.join(row), axis=1).to_frame(name='text')
df_text['key2'] = df['key2_encoded']

In [14]:
df_text[:20]

,text,key2
0,SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1...,18
1,OIL COOLER & LINES HAEIN Coporation_Cheonan NO...,18
2,WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR...,18
3,BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 G...,18
4,SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR E...,18
5,CORE CHARGES FOR CYLINDER PACK AS HAEIN Copora...,18
6,PUMP GP-F TFR-REMAN HAEIN Coporation_Cheonan N...,18
7,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1...,18
8,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...,18
9,GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...,18


In [15]:
import re
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

en_text = []
sentences = list(df_text['text'])
for sen in sentences:
    en_text.append(preprocess_text(sen))

print(df_text.columns.values)

['text' 'key2']


In [16]:
# en_text

In [17]:
y = df_text.key2.values

In [18]:
y[:20]

array([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 45,  4,  4,  4,
       25, 25, 25])

In [19]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [20]:
# !pip uninstall -y tensorflow

In [21]:
# !pip3 install -U "tensorflow==2.11.1"

In [22]:
import tensorflow as tf
print(tf.__version__)
print(tf.__file__)

2.10.0
D:\ShipSupplies\DA\venv\lib\site-packages\tensorflow\__init__.py


In [23]:
def tokenize_text(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
tokenized_text = [tokenize_text(en) for en in en_text]

In [24]:
# en_text[0],tokenized_text[0]

In [25]:
reviews_with_len = [[text, y[i], len(text)] # 토큰화된 text, key값, text 길이
                 for i, text in enumerate(tokenized_text)]
# reviews_with_len[:5]

In [26]:
# 섞고 key2 기준으로 정렬
import random
random.shuffle(reviews_with_len)
reviews_with_len.sort(key=lambda x: x[2])
reviews_with_len[:50]

[[[23365, 3796, 3796, 23365, 3796], 7, 5],
 [[16054, 5658, 4049, 3194, 20965, 14027], 8, 6],
 [[21605, 5658, 4049, 3194, 20965, 13103], 2, 6],
 [[10764, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[9093, 7744, 9347, 29329, 8622, 2112], 48, 6],
 [[5747, 5658, 4049, 3194, 7956, 3796], 10, 6],
 [[11220, 7682, 2373, 3796, 2373, 3796], 4, 6],
 [[7744, 5658, 4049, 3194, 6718, 3847], 24, 6],
 [[3500, 5658, 4049, 3194, 7956, 2132], 50, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 24, 6],
 [[17490, 5658, 4049, 3194, 7956, 2132], 39, 6],
 [[17490, 5658, 4049, 3194, 20965, 13103], 39, 6],
 [[10764, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[10353, 5658, 4049, 3194, 7956, 2132], 19, 6],
 [[16054, 5658, 4049, 3194, 20965, 14027], 8, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[7744, 5658, 4049, 3194, 1042, 10216], 24, 6],
 [[10053, 5658, 4049, 3194, 7956, 3796], 8, 6],
 [[13354, 5658, 4049, 3194, 20965, 13617], 0, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[7744, 5658, 4049, 3194, 1

In [27]:

sorted_text_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_text_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 7), dtype=int32, numpy=
 array([[23365,  3796,  3796, 23365,  3796,     0,     0],
        [16054,  5658,  4049,  3194, 20965, 14027,     0],
        [21605,  5658,  4049,  3194, 20965, 13103,     0],
        [10764,  5658,  4049,  3194,  7956,  2132,     0],
        [ 9093,  7744,  9347, 29329,  8622,  2112,     0],
        [ 5747,  5658,  4049,  3194,  7956,  3796,     0],
        [11220,  7682,  2373,  3796,  2373,  3796,     0],
        [ 7744,  5658,  4049,  3194,  6718,  3847,     0],
        [ 3500,  5658,  4049,  3194,  7956,  2132,     0],
        [ 5009,  5658,  4049,  3194,  7956,  2132,     0],
        [17490,  5658,  4049,  3194,  7956,  2132,     0],
        [17490,  5658,  4049,  3194, 20965, 13103,     0],
        [10764,  5658,  4049,  3194,  7956,  2132,     0],
        [10353,  5658,  4049,  3194,  7956,  2132,     0],
        [16054,  5658,  4049,  3194, 20965, 14027,     0],
        [ 5009,  5658,  4049,  3194,  7956,  2132,     0],
        

In [28]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_text_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [29]:
test_data

<TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [30]:
# for inputs, targets in train_data:
#     print(targets)


In [31]:
TOTAL_BATCHES, TEST_BATCHES

(642, 64)

In [37]:
class TEXT_MODEL(tf.keras.Model):
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        self.embedding = tf.keras.layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.lstm = tf.keras.layers.LSTM(128)
        
        self.pool = tf.keras.layers.GlobalMaxPool1D()
        self.dense_1 = tf.keras.layers.Dense(units=dnn_units, activation="relu")
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = tf.keras.layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = tf.keras.layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3)
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) 
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
                # LSTM 레이어에 입력을 3D 형상으로 변환하여 전달
        lstm_output = self.lstm(tf.expand_dims(concatenated, axis=1))
        
        model_output = self.last_dense(concatenated)
        return model_output

In [38]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 100 #200
CNN_FILTERS = 50 #100
DNN_UNITS = 128 #256
OUTPUT_CLASSES = 61
DROPOUT_RATE = 0.1 # 0.2
NB_EPOCHS = 60
VOCAB_LENGTH
# 100 50 128 61 0.1 10 =>0.88

30522

In [39]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [40]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/60


578/578 [==============================] - 5s 7ms/step - loss: 2.2166 - sparse_categorical_accuracy: 0.4914
Epoch 2/60
578/578 [==============================] - 4s 7ms/step - loss: 1.2136 - sparse_categorical_accuracy: 0.7200
Epoch 3/60
578/578 [==============================] - 4s 7ms/step - loss: 0.9352 - sparse_categorical_accuracy: 0.7755
Epoch 4/60
578/578 [==============================] - 4s 7ms/step - loss: 0.8071 - sparse_categorical_accuracy: 0.8028
Epoch 5/60
578/578 [==============================] - 4s 7ms/step - loss: 0.7348 - sparse_categorical_accuracy: 0.8172
Epoch 6/60
578/578 [==============================] - 4s 7ms/step - loss: 0.6801 - sparse_categorical_accuracy: 0.8259
Epoch 7/60
578/578 [==============================] - 4s 7ms/step - loss: 0.6436 - sparse_categorical_accuracy: 0.8329
Epoch 8/60
578/578 [==============================] - 4s 7ms/step - loss: 0.6104 - sparse_categorical_accuracy: 0.8383
Epoch 9/60
578/578 [==============================] - 4s 7m

In [41]:
results = text_model.evaluate(test_data)
print("60번 학습 : ", results)

64/64 [==============================] - 1s 6ms/step - loss: 1.1091 - sparse_categorical_accuracy: 0.8560
60번 학습 :  [1.1091185808181763, 0.85595703125]


In [ ]:
text_model.fit(train_data, epochs=3)

In [ ]:
results = text_model.evaluate(test_data)
print("13번 학습 : ", results)

In [ ]:
text_model.fit(train_data, epochs=4)

In [ ]:
results = text_model.evaluate(test_data)
print("17번 학습 : ", results)

## pred 는각각 61개 컬럼 라벨에서의 확률

In [42]:
pred = text_model.predict(test_data)

64/64 [==============================] - 1s 4ms/step


In [43]:
pred.shape

(2048, 61)

In [44]:
pred[0]

array([2.45551949e-30, 0.00000000e+00, 2.00791066e-36, 3.90605325e-27,
       4.25406924e-15, 6.34105322e-32, 0.00000000e+00, 1.00000000e+00,
       2.14925038e-27, 4.54142815e-36, 0.00000000e+00, 2.03454402e-34,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.48511584e-37, 0.00000000e+00, 5.04656623e-33, 2.40916340e-16,
       4.13925651e-24, 1.91866206e-27, 4.86637431e-20, 2.08883304e-17,
       6.04481383e-25, 6.79868500e-22, 1.39233295e-18, 8.17786808e-25,
       7.05953202e-25, 0.00000000e+00, 1.40108007e-35, 1.23969656e-30,
       4.67063578e-28, 1.52123629e-26, 6.50525278e-34, 4.67524494e-16,
       0.00000000e+00, 7.94162461e-28, 3.76042795e-35, 1.10912943e-25,
       1.16339720e-30, 0.00000000e+00, 0.00000000e+00, 1.42133718e-20,
       1.11330335e-35, 3.47610873e-17, 1.01790132e-36, 0.00000000e+00,
       1.71528688e-17, 1.18546334e-26, 0.00000000e+00, 3.02889864e-32,
       2.59298747e-18, 2.84224673e-28, 0.00000000e+00, 0.00000000e+00,
      

# predicted_result 50개의 배치(배치 사이즈 : 32) 데이터 마다 예측 라벨링을 리스트에 저장 

In [45]:
predicted_result = []
for j in range(64):
    temp=[]
    for i in range(32) :   
        predicted_class = tf.argmax(pred[i+ j*32]).numpy() ## 가장 높은 확률의 라벨링 데이터를 구함
        temp.append(predicted_class)
    predicted_result.append(temp)
        # print(predicted_class)

In [46]:
for lis in predicted_result:
    print(lis)

[7, 8, 40, 57, 48, 24, 4, 24, 50, 24, 39, 39, 57, 19, 8, 24, 24, 8, 56, 24, 24, 50, 24, 24, 23, 58, 8, 23, 39, 19, 39, 50]
[50, 45, 26, 58, 8, 23, 5, 58, 57, 4, 23, 8, 25, 23, 24, 43, 7, 23, 8, 24, 23, 17, 44, 24, 19, 17, 46, 26, 23, 41, 24, 23]
[24, 8, 39, 24, 44, 4, 26, 46, 4, 0, 8, 46, 39, 48, 7, 54, 4, 44, 50, 23, 8, 39, 17, 57, 19, 26, 17, 57, 44, 24, 24, 24]
[54, 7, 0, 8, 24, 17, 26, 57, 4, 44, 8, 4, 17, 57, 4, 44, 45, 5, 57, 46, 4, 44, 24, 4, 39, 39, 10, 23, 44, 24, 17, 44]
[39, 24, 50, 46, 57, 39, 17, 24, 24, 24, 40, 39, 17, 58, 23, 8, 4, 39, 40, 46, 24, 7, 19, 22, 45, 24, 55, 40, 42, 42, 25, 4]
[24, 58, 24, 42, 42, 40, 10, 58, 50, 57, 4, 4, 57, 20, 24, 40, 51, 25, 58, 58, 45, 43, 26, 40, 51, 8, 57, 0, 24, 48, 25, 24]
[42, 39, 4, 58, 40, 42, 42, 23, 51, 40, 8, 24, 44, 46, 29, 23, 44, 24, 24, 41, 57, 24, 24, 24, 58, 7, 4, 7, 8, 4, 29, 51]
[24, 15, 57, 19, 19, 44, 24, 45, 24, 58, 7, 55, 19, 23, 40, 18, 8, 57, 58, 4, 42, 4, 42, 39, 40, 58, 24, 46, 55, 0, 19, 4]
[4, 39, 43, 46, 57,

# true_result : 테스트 데이터에서의 실제 라벨링

In [47]:
true_result =[]
for inputs, targets in test_data.take(64):
    # 첫 번째 데이터 샘플에 대한 입력(inputs)과 라벨(targets)을 확인

    # Convert EagerTensor to numpy array
    targets_numpy = np.array(targets)

    # Convert numpy array to list
    targets_list = targets_numpy.tolist()
    true_result.append(targets_list)
    # print("Targets:", targets_list)


In [48]:
for lis in true_result:
    print(lis)

[7, 8, 2, 57, 48, 10, 4, 24, 50, 24, 39, 39, 57, 19, 8, 57, 24, 8, 0, 57, 24, 50, 10, 24, 23, 58, 8, 23, 39, 44, 39, 50]
[50, 45, 7, 58, 8, 23, 5, 24, 57, 4, 17, 8, 27, 17, 24, 43, 7, 23, 8, 24, 23, 17, 44, 18, 19, 17, 46, 26, 17, 41, 24, 23]
[40, 8, 39, 40, 44, 24, 7, 46, 4, 0, 8, 46, 39, 48, 7, 54, 4, 44, 50, 23, 8, 39, 17, 57, 19, 7, 17, 52, 44, 24, 24, 24]
[54, 7, 0, 8, 24, 17, 7, 57, 4, 44, 8, 4, 17, 57, 4, 44, 45, 24, 57, 46, 4, 44, 24, 4, 39, 39, 10, 23, 44, 40, 17, 44]
[39, 24, 50, 46, 57, 39, 17, 24, 24, 24, 40, 39, 17, 58, 23, 8, 4, 39, 40, 46, 24, 7, 19, 22, 18, 24, 55, 40, 54, 42, 25, 4]
[24, 58, 24, 42, 42, 40, 10, 58, 50, 57, 4, 4, 57, 20, 24, 40, 51, 25, 58, 58, 42, 43, 24, 40, 51, 8, 57, 0, 24, 48, 25, 24]
[54, 39, 4, 58, 40, 42, 42, 50, 51, 40, 8, 24, 44, 46, 29, 50, 44, 24, 24, 41, 57, 24, 24, 24, 58, 7, 4, 7, 8, 4, 29, 51]
[24, 9, 57, 19, 19, 44, 24, 18, 24, 58, 7, 55, 19, 23, 40, 2, 8, 57, 58, 4, 54, 4, 42, 39, 40, 58, 24, 46, 55, 0, 19, 4]
[4, 39, 43, 46, 57, 24, 4

In [49]:
from sklearn.metrics import f1_score,classification_report

In [50]:
# Calculate F1 score
f1 = f1_score([item for sublist in true_result for item in sublist],
              [item for sublist in predicted_result for item in sublist], average='macro')
print("F1 score:", f1)

F1 score: 0.7877168539613285


In [51]:
print(classification_report([item for sublist in predicted_result for item in sublist],
                            [item for sublist in true_result for item in sublist]))

              precision    recall  f1-score   support

           0       0.88      0.71      0.79        21
           2       0.33      1.00      0.50         3
           3       0.50      1.00      0.67         2
           4       0.99      0.97      0.98       172
           5       0.67      0.50      0.57         4
           7       0.74      1.00      0.85        17
           8       0.99      1.00      0.99        86
           9       0.60      1.00      0.75         3
          10       0.94      1.00      0.97        31
          12       0.52      1.00      0.69        11
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         1
          15       0.80      0.67      0.73         6
          16       1.00      1.00      1.00         5
          17       0.44      0.88      0.58        16
          18       0.69      0.69      0.69        13
          19       0.88      0.90      0.89        62
          20       0.76    

D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
